In [1]:
param_start_date = '2024-02-19'
param_region = 'PMA' #SBA #SKA
param_trip_id = '2024-02-19_SETIA IMAN'

In [2]:
optimization_type = 'Sequence'
max_iterations = 30
no_solutions = 1
max_time = 20
output_type = 'Default'
number_of_locations=10

In [3]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine,MetaData
import urllib
from time import strptime
from sklearn.metrics import DistanceMetric
import pytz
import datetime as dt
from datetime import timedelta

import warnings
warnings.simplefilter(action='ignore')

# import config
import ipynb.fs.full.config as config
# Import Utility Functions
import ipynb.fs.full.functions as functions

from optimize_functions.input_validation import *
from optimize_functions.people import *
from optimize_functions.transform_data import *

from src.optimization.sequence.optimizer import Optimizer
from src.config import Config
from src.optimization.utils.utils import *

In [4]:
## Creating necessary variables
path_optimize = f"{config.path_root}/optimize/raw/"
url = config.url
# spark = SparkSession.builder.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.3").getOrCreate()

In [5]:
# Pipeline should pass the new trip_id if any, to get the correct departure date
backbone_schedule = pd.read_sql_query("SELECT distinct berthing_date, trip_id, trip_id_new, departure_time FROM olv.fact_backbone_schedule where trip_id ='{}';".format(param_trip_id),con=url)
assert len(backbone_schedule) > 0, 'There are no schedules in backbone schedule table. Please check.'
if param_trip_id != "":
    # if backbone_schedule.trip_id_new[0] is not None:
    #     trip_id_new = backbone_schedule.trip_id_new[0]
    #     berthing_date_new = backbone_schedule.berthing_date[0]
    #     param_start_date = berthing_date_new
    # else:
    #     trip_id_new = None
    #     param_start_date = param_trip_id.split("_")[0]
    
    trip_id_new = backbone_schedule.trip_id_new[0]
    param_start_date = backbone_schedule.berthing_date[0].strftime('%Y-%m-%d %H:%M:%S')[:10]
    vname = param_trip_id.split("_")[1]

In [6]:
vmp = functions.vmp(url, param_start_date, optimization_type,vname)

In [7]:
start_date = pd.to_datetime(param_start_date, infer_datetime_format=True)
end_date = start_date + timedelta(days = 15)
end_date = end_date.strftime('%Y-%m-%d')
param_end_date = str(end_date)

In [8]:
full_speed_knots = 5 if (param_region == 'SKA' or param_region == 'SBA') else 8

In [9]:
# to get division_id
vessel_id = pd.read_sql_query(f"select vessel_id from olv.dim_master_vessel where is_active = 1 and vessel_name = '{vname}';",con=url)
vessel_id = vessel_id.vessel_id[0]

param_division_id = pd.read_sql_query(f"select division_id from olv.ref_division_vessels where vessel_id = {vessel_id};",con=url)
param_division_id = param_division_id.division_id[0]

divisions_vessel = pd.read_sql_query("select vessel_id from olv.vw_division_vessels where division_id = {};".format(param_division_id),con=url)
divisions_location = pd.read_sql_query("select location_name from olv.vw_division_locations where division_id = {};".format(param_division_id),con=url)

In [10]:
mb_distance_threshold = 7

In [11]:
# to filter time >= today date
current_time = datetime.now(pytz.timezone('Asia/Kuala_Lumpur')) 
past_days_dt = datetime.strftime(current_time - timedelta(days=10), "%Y-%m-%d")

In [12]:
past_voyage_material = pd.read_sql_query("select * from olv_stg.ds_vessel_voyages_material_d3;",con=url)
past_voyage_material = past_voyage_material.dropna(subset = ['start_loading_time']).reset_index(drop=True)
past_voyage_people = pd.read_sql_query("select * from olv_stg.ds_vessel_voyages_people_d3;",con=url)
past_voyage_people = past_voyage_people.dropna(subset = ['start_loading_time']).reset_index(drop=True)

In [13]:
# Read people demand
people = pd.read_sql_query("SELECT * FROM olv.fact_people_vessel_booking where start_location ='PDJT';",con=url)
people = people.reset_index(drop=True)
people = people.drop(np.array(range(0,len(people))))

In [14]:
# Read Demand Fulfillment data (Bookings)
if param_trip_id != "":

    vessel_details = pd.read_sql_query("SELECT vessel_id, vessel_name from olv.dim_master_vessel", con=url)
    demand_fulfillment = pd.read_sql_query("SELECT * FROM olv.fact_demand_fulfillment_backbone  where voyage_name ='{}';".format(param_trip_id),con=url)

    # split trip_id into vessel_name and voyage date
    demand_fulfillment['vessel_name'] = demand_fulfillment['voyage_name'].str.split('_').str[1]
    demand_fulfillment['start_date'] = param_start_date

    # get vessel_id from vessel_name
    demand_fulfillment = pd.merge(demand_fulfillment, vessel_details, on='vessel_name', how='left')

    # change column name
    demand_fulfillment.rename(columns={'origin':'start_location', 'destination': 'end_location', 'voyage_name': 'trip_id', 
                                       'items': 'container_no', 'material': 'cargo_name'}, inplace=True) 

    # split demand_fulfillment into upperdeck and underdeck # if there is a volume, underdeck, otherwise, upperdeck 
    upperdeck = demand_fulfillment[demand_fulfillment['volume'].isnull() ].reset_index(drop=True)
    underdeck = demand_fulfillment[~(demand_fulfillment['volume'].isnull())].reset_index(drop=True)
    
    upperdeck.rename(columns={'area': 'capacity_booked'}, inplace=True)
    underdeck.rename(columns={'volume':'capacity_booked'}, inplace=True)
        
    assert (len(underdeck) > 0) or (len(upperdeck) > 0), f'There are no demand data for this trip id ({param_trip_id}). Please upload demand fulfillment.'
    
    # Departure time
    if len(backbone_schedule) > 1:
        backbone_schedule.dropna(subset=['departure_time'], inplace=True)
        backbone_schedule.reset_index(drop=True, inplace=True)
        try:
            backbone_schedule.departure_time[0] = backbone_schedule.departure_time[0].strftime("%H:%M:%S")
            backbone_schedule.departure_time = backbone_schedule.departure_time[0][:8]
        except:
            backbone_schedule.departure_time = backbone_schedule.departure_time[0][:8]
        
    assert len(backbone_schedule) == 1, 'There are multiple departure time set for this trip_id. Please ask support from Tech team to check data.'

    if (backbone_schedule.departure_time[0] == None):
        if (param_region ==  'PMA'):
            backbone_schedule.departure_time[0] = '18:00:00'
        else:
            backbone_schedule.departure_time[0] = '00:00:00'
    else:
        if isinstance(backbone_schedule.departure_time[0], str):
            backbone_schedule.departure_time[0] = backbone_schedule.departure_time[0]
        else:
            # backbone_schedule.departure_time[0] = backbone_schedule.departure_time[0].strftime('%H:%M:%S')
            try:
                backbone_schedule.departure_time[0] = backbone_schedule.departure_time[0].strftime("%H:%M:%S")
                backbone_schedule.departure_time = backbone_schedule.departure_time[0][:8]
            except:
                backbone_schedule.departure_time = backbone_schedule.departure_time[0][:8]
   
    # combine start_date and departure_time
    upperdeck['departure_date'] = pd.to_datetime(upperdeck['start_date'] + ' ' + backbone_schedule.departure_time[0])  
    underdeck['departure_date'] = pd.to_datetime(underdeck['start_date'] + ' ' + backbone_schedule.departure_time[0])

    # add +1 day for arrival_date
    upperdeck['arrival_date'] = upperdeck['departure_date'] + timedelta(days=1)
    underdeck['arrival_date'] = underdeck['departure_date'] + timedelta(days=1)

else:
    upperdeck = pd.read_sql_query("""
    SELECT distinct a.*
    FROM olv.fact_upperdeck_demand_fulfillment a
    inner join olv.fact_backbone_schedule b
    on a.trip_id = b.trip_id and a.vessel_id = b.vessel_id
    where b.trip_id_new is null and b.berthing_date >='{}';
    """.format(param_start_date),con=url)

    underdeck = pd.read_sql_query("""
    SELECT distinct a.*
    FROM olv.fact_underdeck_demand_fulfillment a
    inner join olv.fact_backbone_schedule b
    on a.trip_id = b.trip_id and a.vessel_id = b.vessel_id
    where b.trip_id_new is null and b.berthing_date >='{}';
    """.format(param_start_date),con=url)


In [15]:
demand_d3_locations = pd.concat([upperdeck.start_location,upperdeck.end_location,
                                 underdeck.start_location,underdeck.end_location]).unique()
print(demand_d3_locations)

['LARUT' 'KSB' 'BERANTAI']


In [16]:
# Set optimization time to 20 mins if there are 10 or above locations to be visited
if (len(demand_d3_locations) < number_of_locations):
    max_time = 5

In [17]:
# Read Master Locations
locations = pd.read_sql_query("SELECT a.*, b.lunch_start_time ,b.lunch_end_time, b.dinner_start_time, b.dinner_end_time, b.is_food_provision_loading_point \
                                FROM olv.vw_master_location a\
                                JOIN olv.dim_master_location b\
                                ON a.location_name  = b.location_name where a.location_name in {};".format(tuple(demand_d3_locations)),con=url)

mst_loc = locations['location_name'].unique()
locations_doesnt_exist = list(set(demand_d3_locations).difference(mst_loc))
assert len(locations_doesnt_exist) == 0, f'Location {locations_doesnt_exist} missing in master data. Please check if location names(s) are correct (including spacing & CAPS)'

mb_locations = pd.read_sql_query("""
SELECT
    a.*,
    b.lunch_start_time,
    b.lunch_end_time,
    b.dinner_start_time,
    b.dinner_end_time,
    b.is_food_provision_loading_point
FROM
    olv.vw_master_location a
JOIN
    olv.dim_master_location b ON a.location_name = b.location_name
WHERE
    a.location_type IN ('Mooring Buoy')
    AND a.location_region = '{}';
""".format(param_region),con=url)

# assert len(mb_locations) > 0, f'There is no Mooring Buoy information in master data. Please check.'

# Remove inactive mooring buoys
mb_full_list = pd.read_sql_query("SELECT distinct(mooring_buoy_name) FROM olv.fact_mooring_buoy_locations_management;",con=url).mooring_buoy_name
mb_active = pd.read_sql_query("SELECT distinct(mooring_buoy_name) FROM olv.fact_mooring_buoy_locations_management WHERE contract_end_date > '{}';".format(param_start_date),con=url).mooring_buoy_name
mb_inactive = mb_full_list[~mb_full_list.isin(mb_active)]

mb_locations = mb_locations[~mb_locations.location_name.isin(mb_inactive)].reset_index(drop=True)
mb_locations = filter_division(mb_locations, divisions_location, 'location_name')

locations = pd.concat([locations, mb_locations])
locations['lunch_start_time'] = locations['lunch_start_time'].astype('str').str[:8]
locations['lunch_end_time'] = locations['lunch_end_time'].astype('str').str[:8]
locations['dinner_start_time'] = locations['dinner_start_time'].astype('str').str[:8]
locations['dinner_end_time'] = locations['dinner_end_time'].astype('str').str[:8]

In [18]:
# %%
# if param_region == 'PMA':
#     locations.loc[locations.working_end_time > dt.time(20, 00, 00), 'dinner_start_time'] = dt.time(18, 00, 00)
#     locations.loc[locations.working_end_time > dt.time(20, 00, 00), 'dinner_end_time'] = dt.time(20, 00, 00)
#     locations.loc[locations.working_end_time <= dt.time(20, 00, 00), 'dinner_start_time'] = np.nan
#     locations.loc[locations.working_end_time <= dt.time(20, 00, 00), 'dinner_end_time'] = np.nan

In [19]:
# Read priority locations
priority = pd.read_sql_query(f"select * from olv.fact_voyage_schedule_seq_priority where trip_id = '{param_trip_id}'", con=url)

# Check for the number of rows returned
assert len(priority) <= 5, "The existing system can only accommodate up to 5 priority locations"

upperdeck['priority'] = 0
underdeck['priority'] = 0


In [20]:
 # Read master Vessels
vessels = pd.read_sql_query(f"SELECT * FROM olv.vw_master_vessel where vessel_name = '{vname}';",con=url)
vid = vessels.vessel_id.values[0]
original_job_type = vessels['job_type'][0]

# TEMPORARY:  
vessels.expected_vessel_speed_knots = full_speed_knots  
vessels['plug_point_constraint'] = 3 if param_region == 'SKA' else np.nan

vessels['job_type'].mask(vessels['job_type'] == 'Production Operations Supply Run', 'Material Vessel', inplace=True)
vessels['job_type'].mask(vessels['job_type'] == 'Production Project Supply Run', 'Material Vessel', inplace=True)
vessels['job_type'].mask(vessels['job_type'] == 'PSS Material Vessel', 'Material Vessel', inplace=True)
vessels['job_type'].mask(vessels['job_type'] == 'P&A Material Vessel', 'Material Vessel', inplace=True)
vessels['job_type'].mask(vessels['job_type'] == 'T&I Material Vessel', 'Material Vessel', inplace=True)
vessels['job_type'].mask(vessels['job_type'] == 'Drilling Material Vessel', 'Material Vessel', inplace=True)
vessels['job_type'].mask(vessels['job_type'] == 'HUC Material Vessel', 'Material Vessel', inplace=True)

In [21]:
vessels

,vessel_name,vessel_id,vessel_type,region,usable_upper_deck_capacity_m2,fuel_oil_capacity_kl,fuel_consumption_during_voyage_kl,pump_transfer_rate_fuel_oil,fresh_water_capacity_kl,fresh_water_consumption_during_voyage_kl,...,vessel_length,expected_vessel_speed_knots,job_type,captain,dynamic_positioning_2,daily_charter_rate,rating,bollard_pull_strength_tonne,deadweight_tonnage_tonne,plug_point_constraint
0,SETIA IMAN,533000833,AHTS 60,PMA,427.0,430.0,None,150.0,581.0,None,...,None,8,Material Vessel,None,None,None,None,None,None,NaN


In [22]:
# Read Vessel Availability
# D3 material run will consider only Production Operations Supply Run & Passenger Vessel 
vessel_availability = vmp[vmp.vessel_id == vid]
vessel_availability = vessel_availability.drop_duplicates().reset_index(drop=True)
vessel_availability = vessel_availability_optimization_period(vessel_availability,\
                                                                pd.to_datetime(param_start_date, infer_datetime_format=True),
                                                                pd.to_datetime(param_start_date, infer_datetime_format=True) + timedelta(days = 10))

In [23]:
# Update AWB lat long
awb_locations = vmp[vmp.vessel_id.isin(demand_d3_locations)]
awb_locations = awb_locations.drop_duplicates().reset_index(drop=True)
awb_locations = vessel_availability_optimization_period(awb_locations,\
                                                        pd.to_datetime(param_start_date, infer_datetime_format=True),
                                                        pd.to_datetime(param_start_date, infer_datetime_format=True) + timedelta(days = 10))

if len(awb_locations) > 0:
    awb_locations = pd.merge(awb_locations, locations[['location_name','latitude_degree','longitude_degree']], on='location_name',how ='left')
    awb_locations = pd.merge(awb_locations, vessels[['vessel_id','vessel_name']], on='vessel_id', how ='left')
    
    # Replace values in lat long column of locations with values from awb_locations based on vessel_names column
    locations.set_index('location_name', inplace=True)
    locations.update(awb_locations.set_index('vessel_name')['latitude_degree'])
    locations.update(awb_locations.set_index('vessel_name')['longitude_degree'])
    locations.reset_index(inplace=True)

In [24]:
# Read Vessel Maintenance
vessel_maintenance = pd.read_sql_query(f"SELECT * FROM olv.fact_vessel_maintenance where is_active = 1 and vessel_id = '{vid}';",con=url)
vessel_maintenance = vessel_maintenance.drop_duplicates().reset_index(drop=True)
if len(vessel_maintenance) > 0:
    date_ranges = pd.DataFrame()
    for i in vessel_maintenance.index:
        list_dates = pd.date_range(start = vessel_maintenance.maintenance_start_date[i], end = vessel_maintenance.maintenance_end_date[i])
        df_dates = pd.DataFrame(list_dates, columns=['dates'])
        df_dates['ind'] = i
        date_ranges = date_ranges.append(df_dates)

    optimization_period   = pd.date_range(start = pd.to_datetime(param_start_date, infer_datetime_format=True), end = pd.to_datetime(param_start_date, infer_datetime_format=True) + timedelta(days = 10))
    date_ranges = date_ranges[date_ranges.dates.isin(optimization_period)]
    vessel_maintenance = vessel_maintenance.iloc[date_ranges['ind'].unique()]

In [25]:
# Read Fuel Data
fuels_vn = pd.read_sql_query("SELECT vessel_id, cargo_works, standby_normal, anchorage, tied_to_mb, full_speed_knots, a, b, c  FROM olv.ref_vessel_fuel;",con=url)
fuels_vt = pd.read_sql_query("SELECT vessel_type, cargo_works, standby_normal, anchorage, tied_to_mb, full_speed_knots, a, b, c FROM olv.ref_vessel_type_fuel_activity_speed;",con=url)
fuels_vn.full_speed_knots = vessels.loc[vessels.vessel_id == vid,'expected_vessel_speed_knots'][0]
fuels_vt.full_speed_knots = vessels.loc[vessels.vessel_id == vid,'expected_vessel_speed_knots'][0]

In [26]:
# Read Weather data
weather_forecast = pd.read_sql_query("""
SELECT
    b.time,
    a.location_name,
    b.visibility,
    b.wave_direction,
    b.wave_height,
    b.wind_direction,
    b.wind_speed,
    b.updated_date,
    CONCAT(a.location_name, '_', CAST(b.time AS date)) AS id
FROM
    olv.fact_master_weather_forecast b
LEFT JOIN (
    SELECT
        location_name,
        CAST(ROUND(latitude_degree, 2) AS VARCHAR(30)) AS Latitude,
        CAST(ROUND(longitude_degree, 2) AS VARCHAR(30)) AS Longitude
    FROM
        olv.dim_master_location
    GROUP BY
        location_name,
        latitude_degree,
        longitude_degree
) a ON a.Latitude = b.latitude AND a.Longitude = b.longitude
WHERE
    time >= '{}' and time < '{}';""".format(param_start_date, param_end_date),con=url)

assert len(weather_forecast) > 0, f'There is no weather data starting {param_start_date} onwards. Please check.'

In [27]:
# Data Cleaning
past_voyage_material = clean_material_voyage(past_voyage_material)
past_voyage_people = clean_people_voyage(past_voyage_people)
people = clean_people(people)
upperdeck = clean_upperdeck(upperdeck)
underdeck = clean_underdeck(underdeck)
locations = clean_locations(locations)
vessel_availability = clean_vessel_availability(vessel_availability)
vessel_maintenance = clean_vessel_maintenance(vessel_maintenance)
vessels = clean_vessels(vessels)
fuels_vn = clean_fuels(fuels_vn)
fuels_vt = clean_fuels(fuels_vt)
weather_forecast = clean_weather_forecast(weather_forecast)

In [28]:
if len(locations) > 0:
    assert vessels.region.unique()[0] == locations.region.unique()[0], 'Vessel region does not match with locations region. Please check.'

In [29]:
upperdeck

,vessel_id,journey_type,departure_date,arrival_date,origin,destination,material_qty,weight,voyage_id,container_no,priority
0,533000833,Backload,2024-02-19 19:00:00,2024-02-20 19:00:00,LARUT,KSB,0.371612,0.1,2024-02-19_SETIA IMAN,Trash 1,1
1,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,LARUT,2.787091,0.1,2024-02-19_SETIA IMAN,- TRASH BASKET FEL 002,1
2,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,LARUT,5.945795,0.1,2024-02-19_SETIA IMAN,- KEM 235 D,1
3,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,LARUT,7.432243,0.1,2024-02-19_SETIA IMAN,- BASKET AORU 6300503,1
4,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,BERANTAI,0.371612,0.1,2024-02-19_SETIA IMAN,Plug 3,2
5,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,LARUT,1.486449,0.1,2024-02-19_SETIA IMAN,- TANK VST TT 014 (CRW24053),1
6,533000833,Backload,2024-02-19 19:00:00,2024-02-20 19:00:00,LARUT,KSB,0.371612,0.1,2024-02-19_SETIA IMAN,Trash 2,1
7,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,LARUT,5.945795,0.1,2024-02-19_SETIA IMAN,- CONTAINER FDG 3097,1
8,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,LARUT,2.787091,0.1,2024-02-19_SETIA IMAN,- TRASH BASKET FEL 057,1
9,533000833,Backload,2024-02-19 19:00:00,2024-02-20 19:00:00,LARUT,KSB,0.371612,0.1,2024-02-19_SETIA IMAN,Trash 3,1


In [30]:
underdeck

,vessel_id,journey_type,departure_date,arrival_date,origin,destination,material_type,material_qty,voyage_id,priority
0,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,LARUT,Fuel Oil,25.0,2024-02-19_SETIA IMAN,1
1,533000833,Payload,2024-02-19 19:00:00,2024-02-20 19:00:00,KSB,LARUT,Fresh Water,25.0,2024-02-19_SETIA IMAN,1


In [31]:
# ASSERTION:
if (len(upperdeck) != 0) | (len(underdeck) != 0):

    # Transform Master Data
    vessels, locations, vessel_availability, distance_matrix, mb_for_location = transform_master_data(vessels, fuels_vn, fuels_vt, vessel_availability, vessel_maintenance,locations, param_region, COMMODITIES, UOM_CONVERSION, UOM, MILES_CONVERTER,mb_distance_threshold)
    bookings_d3, past_voyages_d3 = transform_sequence(upperdeck, underdeck, people, locations, past_voyage_material, past_voyage_people)

    # Input validation
    input_validation_sequence(vessels, vessel_availability, locations, bookings_d3, fuels_vn, fuels_vt,param_region)

    # Add departure time for each trip id
    bookings_d3 = pd.merge(bookings_d3, backbone_schedule, how='left', left_on='voyage_id', right_on='trip_id').drop(columns='trip_id')

    # Check if vessels in demand fulfilment exist in master vessels as 'Material Vessel'
    vessels_allocated = pd.concat([upperdeck,underdeck]).vessel_id.unique()
    intersect = set(vessels_allocated).intersection(vessels[vessels.job_type == 'Material Vessel'].vessel_id.unique())
    assert len(intersect) == len(vessels_allocated), "The allocated vessel(s) in demand fulfillment does not belong to the selected job type."

    # Check if vessels in demand fulfilment exist in vessel_availability
    intersect = set(vessels_allocated).intersection(vessel_availability.vessel_id.unique())
    assert len(intersect) == len(vessels_allocated), "The vessel allocated is not available for this period"

    # Check if origin == destination
    if len(upperdeck) != 0 : assert all(upperdeck.origin != upperdeck.destination), "There are upperdeck demand with the same origin and destination"
    if len(underdeck) != 0 : assert all(underdeck.origin != underdeck.destination), "There are underdeck demand with the same origin and destination"

    # Check if one container for multiple locations
    # multi_loc = upperdeck.groupby(['voyage_id', 'container_no','journey_type'], as_index=False)['destination'].nunique()
    # multi_loc = multi_loc[~multi_loc.container_no.str.contains('Loose|loose|N/A')]
    # if any(multi_loc.destination > 1) : print(multi_loc[multi_loc.destination > 1])
    # assert all(multi_loc.destination == 1), "There are demand where one container is for multiple locations"

    # To remove all rows for past voyages - (this shouldnt be here. need further investigation)
    past_voyages_d3 = past_voyages_d3.reset_index(drop=True)
    past_voyages_d3 = past_voyages_d3.drop(np.array(range(0,len(past_voyages_d3))))
    # past_voyages_d3

    # print(bookings_d3)
    
    vessel_availability = vessel_availability[~(vessel_availability.availability_start_date > param_start_date)]
    vessel_availability



In [32]:
vessels['expected_vessel_speed'].fillna(value=8, inplace=True)

In [33]:
bookings_d3

,vessel_id,region,journey_type,origin,destination,material_type,voyage_id,week,departure_date,arrival_date,material_qty,weight,container_no,priority,berthing_date,trip_id_new,departure_time
0,533000833,PMA,Backload,LARUT,KSB,upperdeck,2024-02-19_SETIA IMAN,2024_07,2024-02-19 19:00:00,2024-02-20 19:00:00,1.114836,0.0003,3,1,2024-02-19 19:00:00,2024-02-19_SETIA IMAN,19:00:00
1,533000833,PMA,Payload,KSB,BERANTAI,upperdeck,2024-02-19_SETIA IMAN,2024_07,2024-02-19 19:00:00,2024-02-20 19:00:00,1.114836,0.0003,3,2,2024-02-19 19:00:00,2024-02-19_SETIA IMAN,19:00:00
2,533000833,PMA,Payload,KSB,LARUT,upperdeck,2024-02-19_SETIA IMAN,2024_07,2024-02-19 19:00:00,2024-02-20 19:00:00,45.708296,0.0009,9,1,2024-02-19 19:00:00,2024-02-19_SETIA IMAN,19:00:00
3,533000833,PMA,Payload,KSB,LARUT,fuel,2024-02-19_SETIA IMAN,2024_07,2024-02-19 19:00:00,2024-02-20 19:00:00,25.000000,0.0000,0,1,2024-02-19 19:00:00,2024-02-19_SETIA IMAN,19:00:00
4,533000833,PMA,Payload,KSB,LARUT,water,2024-02-19_SETIA IMAN,2024_07,2024-02-19 19:00:00,2024-02-20 19:00:00,25.000000,0.0000,0,1,2024-02-19 19:00:00,2024-02-19_SETIA IMAN,19:00:00


In [ ]:
# to store interplatform info
interplatform_df = bookings_d3[(bookings_d3["region"]==param_region) & (bookings_d3["journey_type"]=="Interplatform")]

In [34]:
# # - To support Bridgelink location type
# platforms = pd.read_sql_query("SELECT location_group, location_name \
#                                 FROM olv.dim_master_location where SUBSTRING(location_type, 1, 19) = 'Bridgelink Platform' and  \
#                                 location_name in {} group by location_group, location_name;".format(tuple(demand_d3_locations)),con=url)

# bookings_d3 = bridgelink_filtered(bookings_d3, platforms)

# bridgelink_platform_list = get_bridgelink_platform_list(platforms)
# for bridgelink_platform_element in bridgelink_platform_list:
#     locations.replace(bridgelink_platform_element[1:],bridgelink_platform_element[0], inplace=True)
    
#     for element in bridgelink_platform_element[1:]:
#         mb_for_location.replace(element,bridgelink_platform_element[0], inplace=True)
#         distance_matrix.columns = distance_matrix.columns.str.replace(element,bridgelink_platform_element[0])
#         distance_matrix.rename(index={element:bridgelink_platform_element[0]}, inplace=True)


In [35]:
mb_for_location

""


In [36]:
if len(bookings_d3) != 0:

    if pd.isnull(backbone_schedule.departure_time[0]) == False:
        berthing_hour = str(backbone_schedule.departure_time[0])
        bookings_d3.departure_date = pd.to_datetime(param_start_date + " " + berthing_hour, infer_datetime_format=True)
  
    region = param_region
    # To support cross land location 
    update_distance_matrix_for_cross_land_locations(region, distance_matrix, locations, url)
    start_date = pd.to_datetime(param_start_date, infer_datetime_format=True)

    config = Config(optimization_type, output_type, region, start_date, None, None, max_iterations, max_time)

    optimizer = Optimizer(config)
    material_global_output, people_global_output, material_vessel_voyage, people_vessel_voyage, \
    material_allocation, people_allocation, activity = optimizer.run(locations, bookings_d3, past_voyages_d3, vessels, vessel_availability, distance_matrix, weather_forecast, mb_for_location, priority)

    # Process Output
    final_material_global_output, final_people_global_output, final_material_vessel_voyage, \
    final_people_vessel_voyage, final_material_allocation, final_people_allocation, final_activity = transform_d3_output(material_global_output, people_global_output, material_vessel_voyage, \
    people_vessel_voyage, material_allocation, people_allocation,activity)

    # to check whether the final_activity fulfills interplatform ranking
    if not interplatform_df.empty:
        sequence_df = final_activity[['sequence','origin_location']].drop_duplicates().reset_index(drop=True)
        sequence_dict = sequence_df.set_index('origin_location')['sequence'].to_dict()

        interplatform_df['origin_sequence'] = interplatform_df['origin'].map(sequence_dict)
        interplatform_df['destination_sequence'] = interplatform_df['destination'].map(sequence_dict)

        # Filter rows where destination_sequence comes before origin_sequence
        result_df = interplatform_df[interplatform_df['origin_sequence'] > interplatform_df['destination_sequence']]

        assert result_df.empty, "Interplatform sequence error. Please rerun D3."

    # Data transformation to handle PLT integration (remove MB)
    vessel_voyages_column_names = pd.read_sql_query(f"SELECT name FROM sys.columns WHERE object_id = OBJECT_ID('olv.fact_backbone_schedule_plt_d3');",con=url)
    vessel_voyages_column_names = vessel_voyages_column_names.name.tolist()
    
    final_material_vessel_voyage_plt = final_material_vessel_voyage[['vessel_id','vessel_name','origin_location','destination_location', 'sequence',
                                                                    'end_date','start_date','voyage_id','job_type','start_moving_time','end_moving_time']]

    final_material_vessel_voyage_plt.rename(columns = {'origin_location':'start_location','destination_location': 'destination',
        'end_date':'tar_arrival_date','start_date':'tar_dept_date','sequence':'seq',
        'voyage_id':'trip_id','end_moving_time':'eta_arrival_date','start_moving_time':'eta_dept_date'}, inplace = True)
    
    add_columns = [x for x in vessel_voyages_column_names if x not in final_material_vessel_voyage_plt.columns]
    final_material_vessel_voyage_plt = final_material_vessel_voyage_plt.reindex(list(final_material_vessel_voyage_plt)+add_columns, axis=1)
    final_material_vessel_voyage_plt = final_material_vessel_voyage_plt[vessel_voyages_column_names]
    final_material_vessel_voyage_plt['berthing_date'] = param_start_date
    final_material_vessel_voyage_plt['trip_id_new'] = trip_id_new if trip_id_new is not None else param_trip_id
    final_material_vessel_voyage_plt['is_active'] = 1
    final_material_vessel_voyage_plt['is_processed'] = 1

    final_material_vessel_voyage_plt = pd.merge(final_material_vessel_voyage_plt, locations[['location_id','location_type']], \
        left_on='start_location', right_on= 'location_id',how ='left').drop(columns='location_id')
    final_material_vessel_voyage_plt.rename(columns = {'location_type':'origin_location_type'}, inplace = True)
    
    final_material_vessel_voyage_plt = pd.merge(final_material_vessel_voyage_plt, locations[['location_id','location_type']], \
        left_on='destination', right_on= 'location_id',how ='left').drop(columns='location_id')
    final_material_vessel_voyage_plt.rename(columns = {'location_type':'destination_location_type'}, inplace = True)
                                                       
    # Iterate over the rows and update the values in the column
    for i in range(len(final_material_vessel_voyage_plt)):
        if final_material_vessel_voyage_plt.at[i, 'origin_location_type'] == 'Mooring Buoy':
            # Check if it's not the last row to avoid an IndexError
            if i < len(final_material_vessel_voyage_plt) - 1:
                final_material_vessel_voyage_plt.at[i, 'start_location'] = final_material_vessel_voyage_plt.at[i + 1, 'start_location']
        if final_material_vessel_voyage_plt.at[i, 'destination_location_type'] == 'Mooring Buoy':
            # Check if it's not the last row to avoid an IndexError
            if i < len(final_material_vessel_voyage_plt) - 1:
                final_material_vessel_voyage_plt.at[i, 'destination'] = final_material_vessel_voyage_plt.at[i + 1, 'destination']
                final_material_vessel_voyage_plt.at[i, 'eta_arrival_date'] = final_material_vessel_voyage_plt.at[i + 1, 'eta_arrival_date']
                        
    # Drop rows where origin and destination are the same
    final_material_vessel_voyage_plt = final_material_vessel_voyage_plt[final_material_vessel_voyage_plt["start_location"] != final_material_vessel_voyage_plt["destination"]]
    # final_material_vessel_voyage_plt = transform_plt(final_material_vessel_voyage_plt,mb_for_location)
    final_material_vessel_voyage_plt["seq"] = range(1, len(final_material_vessel_voyage_plt) + 1)
    
    final_material_vessel_voyage_plt = final_material_vessel_voyage_plt.drop(columns=['origin_location_type', 'destination_location_type'])
    
    current_time = datetime.now(pytz.timezone('Asia/Kuala_Lumpur'))
    current_dt = datetime.strftime(current_time, "%Y-%m-%d %H:%M:%S")
    final_material_vessel_voyage_plt["updated_date"] = current_dt
    final_material_vessel_voyage_plt["updated_date"] = pd.to_datetime(final_material_vessel_voyage_plt["updated_date"])
    final_material_vessel_voyage_plt = final_material_vessel_voyage_plt.drop(columns=['fact_backbone_schedule_id'])

    # add division_id to table
    final_material_vessel_voyage['division_id'] = param_division_id
    final_material_allocation['division_id'] = param_division_id
    final_people_vessel_voyage['division_id'] = param_division_id
    final_people_allocation['division_id'] = param_division_id
    
    # create new column job_subtype to store original_job_type
    dfs = [final_material_vessel_voyage_plt, final_material_global_output, final_people_global_output, 
        final_material_vessel_voyage, final_people_vessel_voyage, final_material_allocation, 
        final_people_allocation, final_activity]
    
    # Iterate over the dataframes
    for df in dfs:
        if df is not None and 'job_type' in df.columns:
            # Create new column job_subtype 
            df['job_subtype'] = original_job_type
            
    # Define the delete query
    delete_query = f"DELETE FROM olv.fact_backbone_schedule_plt_d3 WHERE trip_id = '{param_trip_id}'"
    # Execute the delete query
    with url.begin() as connection:
        connection.execute(delete_query)

    final_material_vessel_voyage_plt.to_sql('fact_backbone_schedule_plt_d3', con=url, if_exists='append',index=False, schema="olv")

    # Write to DB
    with url.begin() as conn:
        truncate_query1 = sqlalchemy.text("TRUNCATE TABLE olv_stg.ds_global_output_material_d3") 
        conn.execute(truncate_query1)
        truncate_query2 = sqlalchemy.text("TRUNCATE TABLE olv_stg.ds_global_output_people_d3") 
        conn.execute(truncate_query2)
        truncate_query3 = sqlalchemy.text("TRUNCATE TABLE olv_stg.ds_vessel_voyages_material_d3") 
        conn.execute(truncate_query3)
        truncate_query4 = sqlalchemy.text("TRUNCATE TABLE olv_stg.ds_vessel_voyages_people_d3") 
        conn.execute(truncate_query4)
        truncate_query5 = sqlalchemy.text("TRUNCATE TABLE olv_stg.ds_demand_allocation_d3") 
        conn.execute(truncate_query5)
        truncate_query6 = sqlalchemy.text("TRUNCATE TABLE olv_stg.ds_passenger_allocation_d3") 
        conn.execute(truncate_query6)
        truncate_query7 = sqlalchemy.text("TRUNCATE TABLE olv_stg.ds_activity_d3") 
        conn.execute(truncate_query7)

    final_material_global_output.to_sql('ds_global_output_material_d3', con=url, if_exists='append',index=False, schema="olv_stg")
    final_people_global_output.to_sql('ds_global_output_people_d3', con=url, if_exists='append',index=False, schema="olv_stg")
    final_material_vessel_voyage.to_sql('ds_vessel_voyages_material_d3', con=url, if_exists='append',index=False, schema="olv_stg")
    final_people_vessel_voyage.to_sql('ds_vessel_voyages_people_d3', con=url, if_exists='append',index=False, schema="olv_stg")
    final_material_allocation.to_sql('ds_demand_allocation_d3', con=url, if_exists='append',index=False, schema="olv_stg")
    final_people_allocation.to_sql('ds_passenger_allocation_d3', con=url, if_exists='append',index=False, schema="olv_stg")
    final_activity.to_sql('ds_activity_d3', con=url, if_exists='append',index=False, schema="olv_stg")


None
Type of job is None
available_vessels:  ['533000833']
windows_by_location: {}
Best solution. Feasible: True, cost: 43351.2
voyage.vessel.full_engine_speed: 8
activity.speed: 8
voyage.vessel.full_engine_speed: 8
activity.speed: 5
voyage.vessel.full_engine_speed: 8
activity.speed: 8
Material - Total cost for solution: 43351.164438609405
People - Total cost for solution: 0
